In [1]:
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load data
transform = transforms.Compose([transforms.ToTensor(),

                                ])

trainset = torchvision.datasets.MNIST(root='./data/', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data/', train=False, download=True, transform=transform)


class MNISTDataset(Dataset):

  def __init__(self, X, y):

    self.X = X.type(torch.cuda.FloatTensor).to(device)
    self.y = nn.functional.one_hot(y).type(torch.cuda.FloatTensor).to(device)


  def __len__(self):
    return len(self.X)


  def __getitem__(self, idx):

    features = self.X[idx]
    target = self.y[idx]

    return features, target


train_dataset = MNISTDataset(trainset.data, trainset.targets)
test_dataset = MNISTDataset(testset.data, testset.targets)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=32)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=32)


In [ ]:
train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze().to("cpu")
label = train_labels[0].to("cpu")
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label.argmax().item()}")




Feature batch shape: torch.Size([32, 28, 28])
Labels batch shape: torch.Size([32, 10])


In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()

        # Convolutional layer 1
        self.conv1 = nn.Conv2d(1, 32, 3, 1)

        # Convolutional layer 2
        self.conv2 = nn.Conv2d(32, 64, 3, 1)

        # Fully connected layer 1
        self.fc1 = nn.Linear(1600, 1200)

        # Fully connected layer 2
        self.fc2 = nn.Linear(1200, 10)

    def forward(self, x):
        # Pass the input image through the convolutional layers
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, 2)

        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, 2)

        # Flatten the output from the convolutional layers
        x = x.view(-1, 1600)

        # Pass the output from the convolutional layers through the fully connected layers
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)

        # Return the output of the fully connected layers
        return x

In [ ]:
model = MyModel().to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 100


print("So it begins")
model.train()
for epoch in tqdm(range(n_epochs)):
    for X_batch, y_batch in train_loader:

        X_batch = X_batch.unsqueeze(1)
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Finished epoch {epoch}, latest loss {loss}')


In [2]:
%pip install torchmetrics
import torchmetrics

   ---------------------------------------- 0.0/931.6 kB ? eta -:--:--
   ---------------------------------------- 931.6/931.6 kB 6.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Initialize precision and recall metrics
precision_metric = torchmetrics.Precision(task="multiclass", num_classes=10).to(device)
recall_metric = torchmetrics.Recall(task="multiclass", num_classes=10).to(device)
accuracy_metric = torchmetrics.Accuracy(task="multiclass", num_classes=10).to(device)



# Iterate over the test data
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        # Make predictions
        X_batch = X_batch.unsqueeze(1)
        outputs = model(X_batch)

        # Update accuracy metric

        _, predicted = torch.max(outputs.data, 1)
        _, y_batch = torch.max(outputs.data, 1)

        accuracy_metric.update(predicted, y_batch)
        precision_metric.update(predicted, y_batch)
        recall_metric.update(predicted, y_batch)




# Calculate average precision and recall
accuracy = precision_metric.compute()
precision = precision_metric.compute()
recall = recall_metric.compute()
print('Accuracy on test data: %f' % accuracy)
print('Precision on test data: %f' % precision)
print('Recall on test data: %f' % recall)